In [ ]:
#| default_exp learner

# Learner and Callbacks

> Defines custom fast.ai Callbacks for Indic-CLIP training, specifically for calculating retrieval metrics during validation.

## Colab Setup

In [ ]:
#| hide
# Mount Google Drive (Optional, but recommended for persistent storage)
from pathlib import Path
import sys
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
    # Define PROJECT_ROOT for Colab
    PROJECT_ROOT = Path('/content/drive/MyDrive/Indic-Clip') # Adjust path if needed
    if not PROJECT_ROOT.exists():
        print(f"Warning: Project directory not found at {PROJECT_ROOT}. Please ensure it exists.")
    else:
        # Add project root to sys.path
        if str(PROJECT_ROOT) not in sys.path:
            sys.path.insert(0, str(PROJECT_ROOT))
            print(f"Added {PROJECT_ROOT} to sys.path")
        # Change current working directory
        os.chdir(PROJECT_ROOT)
        print(f"Changed working directory to: {os.getcwd()}")

except ModuleNotFoundError:
    print("Not running in Colab, skipping Drive mount.")
    # Define PROJECT_ROOT for local execution (adjust if needed)
    PROJECT_ROOT = Path.cwd()
    if PROJECT_ROOT.name == 'nbs': PROJECT_ROOT = PROJECT_ROOT.parent
    print(f"Running locally. Project root assumed: {PROJECT_ROOT}")
    if str(PROJECT_ROOT) not in sys.path:
         sys.path.insert(0, str(PROJECT_ROOT))
         print(f"Added {PROJECT_ROOT} to sys.path")
except Exception as e:
    print(f"An error occurred during Colab setup: {e}")
    PROJECT_ROOT = Path('.').resolve()
    print(f"Defaulting project root to current dir: {PROJECT_ROOT}")

Mounted at /content/drive
Google Drive mounted successfully.
Added /content/drive/MyDrive/Indic-Clip to sys.path
Changed working directory to: /content/drive/MyDrive/Indic-Clip


In [ ]:
#| export
# Use try-except for robustness, especially during development/export
try:
    import indic_clip.core
    print("Imported indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive detected, setting PROJECT_ROOT to /content/drive/MyDrive/Indic-Clip
Ensure your project files are located there.
Imported indic_clip.core


In [ ]:
#| hide
# !pip install -qr requirements.txt
# !pip install scikit-learn # Make sure sklearn is installed for metrics

In [ ]:
#| hide
# from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torch.distributed as dist
import numpy as np
import logging

from fastai.callback.core import Callback
from fastai.learner import Learner
from fastai.data.load import DataLoader
from fastcore.all import *
from fastai.torch_core import * # For tensor()
from fastai.text.all import *

try:
    from indic_clip.core import get_logger
    from indic_clip.evaluation.metrics import calculate_retrieval_metrics, calculate_zeroshot_accuracy # Import actual functions
    from indic_clip.loss import ContrastiveLoss
except ModuleNotFoundError:
    print("Could not import project modules in 09_learner.ipynb.")
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    # Define stubs if import fails
    def calculate_retrieval_metrics(*args, **kwargs): return {'mean_recall': 0.0, 'i2t_r@1': 0.0, 't2i_r@1': 0.0}
    def calculate_zeroshot_accuracy(*args, **kwargs): return 0.0

logger = get_logger(__name__)

Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core


## Retrieval Metric Callback

In [ ]:
#| export
# Removed the stub function as we should import the actual one from evaluation.metrics
# If 11_evaluation_metrics.ipynb is not yet implemented, the try-except block above will define a basic stub.

In [ ]:
#| export

_retrieval_metric_values = {}

class RetrievalMetricCallback(Callback):
    """
    1) Collects features during validation in `after_pred`.
    2) Computes retrieval metrics in `after_validate`.
    3) Stores results in `_retrieval_metric_values` for placeholder metrics.
    """
    order = Recorder.order - 1 # Run before Recorder to get preds, but after model

    def __init__(self, k_values=[1, 5, 10]):
        store_attr()
        self.img_feats_val = [] # Use instance attributes for collection
        self.txt_feats_val = []

    def before_validate(self):
        """Initialize feature lists before validation starts."""
        self.img_feats_val = []
        self.txt_feats_val = []
        logger.debug("RetrievalMetricCallback: Initialized lists for validation features.")

    def after_pred(self):
        """If validating, collect features from learn.pred."""
        # Check context: learn.training, learn.model.training, learn.dls.valid is active?
        if not self.training and self.learn.pred is not None:
            # self.learn.pred is the tuple (img_feat, txt_feat, logit_scale) from model.forward
            try:
                img_feat, txt_feat, _ = self.learn.pred # logit_scale not needed for metrics here
                # Collect features on CPU to free GPU memory
                self.img_feats_val.append(img_feat.cpu())
                self.txt_feats_val.append(txt_feat.cpu())
            except Exception as e:
                 logger.error(f"Error collecting features in RetrievalMetricCallback.after_pred: {e}", exc_info=True)

    # _gather_features method remains the same as provided before

    def _gather_features(self, features_list):
        "Gathers features from all GPUs if in distributed mode."
        # Ensure tensors are detached before gathering if they aren't already
        detached_list = [t.detach() for t in features_list]

        if dist.is_available() and dist.is_initialized():
            world_size = dist.get_world_size()
            if world_size <= 1: return detached_list

            logger.debug(f"Gathering features from {world_size} GPUs.")
            gathered_nested = [None] * world_size
            try:
                dist.all_gather_object(gathered_nested, detached_list)
                all_features = []
                for rank_list in gathered_nested:
                    all_features.extend(rank_list)
                logger.debug(f"Gathered total {len(all_features)} feature tensors.")
                return all_features
            except Exception as e:
                logger.error(f"Error during all_gather_object: {e}", exc_info=True)
                return []
        else:
            return detached_list


    def after_validate(self):
        """Concatenate features, compute metrics, store in _retrieval_metric_values."""
        global _retrieval_metric_values # Modify the shared dict

        logger.debug("RetrievalMetricCallback: Starting metric calculation in after_validate.")

        if not self.img_feats_val or not self.txt_feats_val:
            logger.warning("No features collected during validation. Skipping retrieval metric calculation.")
            _retrieval_metric_values = {} # Clear previous results
            return

        logger.debug(f"Collected {len(self.img_feats_val)} img batches, {len(self.txt_feats_val)} txt batches.")

        # --- Gather ---
        all_img_features_gathered = self._gather_features(self.img_feats_val)
        all_txt_features_gathered = self._gather_features(self.txt_feats_val)

        # --- Calculate (only on rank 0) ---
        is_rank_zero = not (dist.is_available() and dist.is_initialized()) or dist.get_rank() == 0
        calculated_metrics = {}
        if is_rank_zero:
            if not all_img_features_gathered or not all_txt_features_gathered:
                logger.warning(f"Rank 0: No features after gathering. Skipping.")
            else:
                try:
                    device = next(self.learn.model.parameters()).device
                    img_feats_all = torch.cat(all_img_features_gathered).to(device)
                    txt_feats_all = torch.cat(all_txt_features_gathered).to(device)
                    logger.debug(f"Rank 0 concatenated features. Img: {img_feats_all.shape}, Txt: {txt_feats_all.shape}")

                    # Get Logit Scale (ensure model is on correct device)
                    self.learn.model.to(device)
                    if hasattr(self.learn.model, 'logit_scale'):
                        with torch.no_grad():
                            logit_scale = self.learn.model.logit_scale.exp().to(device) # Get current exp value
                    else:
                        logit_scale = torch.tensor(np.exp(np.log(1 / 0.07)), device=device)

                    # Calculate Metrics
                    calculated_metrics = calculate_retrieval_metrics(img_feats_all, txt_feats_all, logit_scale, self.k_values)
                    logger.info(f"Rank 0 calculated retrieval metrics: {calculated_metrics}")

                except Exception as e:
                    logger.error(f"Error during metric calculation on rank 0: {e}", exc_info=True)
                    calculated_metrics = {}

        # --- Update shared dict and clean up ---
        _retrieval_metric_values = calculated_metrics # Update shared dict
        self.img_feats_val = [] # Clean up instance lists
        self.txt_feats_val = []
        logger.debug(f"Cleaned up feature lists on rank {dist.get_rank() if dist.is_initialized() else 0}")

## Example Usage Notes

In [ ]:
#| eval: false

# How to use this callback with a fastai Learner:

# from fastai.learner import Learner
# from fastai.optimizer import Adam
# from fastai.callback.wandb import WandbCallback
# from fastai.callback.schedule import fit_one_cycle
# from fastai.callback.fp16 import MixedPrecision
# from fastai.callback.progress import ProgressCallback
# from fastai.callback.core import SaveModelCallback

# # Assume:
# # - 'dls' is your DataLoaders object from 04_data_creation
# # - 'model' is your IndicCLIP model instance from 07_model_clip
# # - 'loss_func' is your ContrastiveLoss instance from 08_loss

# # Instantiate the callback
# retrieval_cb = RetrievalMetricCallback(k_values=[1, 5, 10])

# # Add it to your list of callbacks
# cbs = [
#     # Other essential callbacks
#     MixedPrecision(), # If using AMP
#     ProgressCallback(),
#     # WandbCallback(log_preds=False, log_model=True), # Ensure WandB is initialized
#     SaveModelCallback(monitor='valid_mean_recall', comp=np.greater), # Monitor metric logged by RetrievalMetricCallback

#     # Your custom callback
#     retrieval_cb
# ]

# # Create the Learner
# # Make sure to pass the metric function (even a dummy one) to Learner for the name to be registered
# # fastai needs a function reference in metrics list for SaveModelCallback monitor to work correctly
# def valid_mean_recall(logits=None, targets=None): return 0.0 # Dummy metric fn
# learn = Learner(dls, model, loss_func=loss_func, opt_func=Adam, cbs=cbs, metrics=[valid_mean_recall])

# # Start training
# # learn.fit_one_cycle(n_epoch=5, lr_max=1e-4)

# # During validation epochs, RetrievalMetricCallback will compute and log:
# # valid_i2t_r_at_1, valid_t2i_r_at_5, valid_mean_recall etc. via learn.recorder.log
# # These will appear in WandB logs (picked up by WandbCallback's after_epoch) and can be used by SaveModelCallback.

In [ ]:
#| hide
# Run this in terminal if using nbdev
if __name__ == '__main__' and '__file__' not in globals(): # Only run if executing as script
    try:
        import nbdev.export
        nbdev.export.nb_export('09_learner.ipynb', lib_path='./indic_clip', name='learner')
        print("Exported 09_learner.ipynb to indic_clip/learner.py")
    except ImportError:
        print("nbdev not found. Run 'pip install nbdev' and then 'nbdev_export' in terminal.")
    except Exception as e:
        print(f"Error exporting notebook: {e}")